# Simulation Calibration

Simulation calibration tries to make the rendered camera image as similar as possible to the real one.
This should maximize the transferability of learned policies.

This performs eye-in-hand calibration. It does this via comparing segmentation masks of real and rendered images using differential evolution.

There is a classical way of doing eye-in-hand calibration using robot trajectories and visual trajectories from calibration patterns. Ask Nikolaus about that.

## Image Based calibration

Only run the first cell once.

In [ ]:
import types
import os.path as osp

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import pybullet as p

from gym_grasping.envs.robot_sim_env import RobotSimEnv

plt.rcParams['figure.figsize'] = [10, 5]


#image_size = (84, 84)  # calibrate small res
image_size = (480, 480)  # calibrate high res

env = RobotSimEnv(task='stackVel', renderer='egl', max_steps=200,
                  initial_pose='close', gripper_delay=0,
                  gripper_act_type="continuous",
                  param_randomize=False,
                  img_size=image_size)

# Clean up the env, by removing non robot stuff
for objIds in (env._task.objects, env._task.surfaces):
    print(objIds)
    for objId in objIds:
        p.removeBody(objId, physicsClientId=env.cid)

assert env.robot.gripper.act_type == 'continuous'

# Load image from file

In [ ]:
image_fn = "./data/kuka_iiwa/"
image = Image.open("/home/kuka/lang/robot/gym_grasping/gym_grasping/calibration/data/kuka_iiwa/2019_02_05.png")

# Load image from Realsense

In [ ]:
import cv2
import time
from robot_io.cams.realsenseSR300_librs2 import RealsenseSR300
cam = RealsenseSR300()
#from robot_io.kuka_iiwa.wsg50_controller import WSG50Controller
#import time
#gripper = WSG50Controller(max_opening_width=77)
#gripper.open_gripper()


In [ ]:
time.sleep(1)
image, _ = cam.get_image(flip_image=True, crop=True)  # size 480x480

assert image.shape == (480, 480, 3)
if image_size != (480,480):
    image = cv2.resize(image, image_size)

In [ ]:
save_image = False
if save_image:
    from PIL import Image
    img = Image.fromarray(image)
    img_path = "/home/kuka/lang/robot/gym_grasping/gym_grasping/calibration/data/kuka_iiwa/2020_10_28.png"
    img.save(img_path)
    print("saved image.")

path_old = "/home/kuka/lang/robot/gym_grasping/gym_grasping/calibration/data/kuka_iiwa/2020_10_26.png"
img_old = Image.open(path_old)
#with open(path_old, "r") as fo:
#    img_old = Image.open(fo)

In [ ]:
mix = (np.array(img_old) * .5 + image * .5).round().astype(np.uint8)
plt.imshow(mix)
plt.show()

In [ ]:
#env.reset()
render = env.render()

assert render.shape[:2] == image_size
fig, ax = plt.subplots(1,2)
for x in ax: x.set_axis_off()
fig.suptitle("Default Views (different aspect ratio)")
ax[0].set_title("Camera Image")
ax[0].imshow(image)
ax[1].set_title("Env Image")
ax[1].imshow(render)
plt.show()


# Threshold Camera Image

In order to do a robust comparison of real image and rendered image, first extract segmentation masks of the gripper by thresholding the image. This is done by an interactive plot (but can be turned off).

In [ ]:
from ipywidgets import interact

do_threshold = False

dist = np.linalg.norm(np.array(image), axis=2)

if do_threshold == False:
    t = 343
    y = 280
    
    masked_image = np.array(image)
    masked_image[dist > t] = (255, 255, 255)
    masked_image[y:,:] = (255, 255, 255)
    
    # set this once, don't modify
    img_mask = dist < t
    img_mask_cache = img_mask.copy()
    
    fig, ax = plt.subplots(1,1)
    ax.set_title("Threshold Result")
    ax.set_axis_off()
    ax = plt.imshow(masked_image)
    plt.show()
    
else:
    %matplotlib notebook
    import numpy as np
    from math import pi
    import matplotlib.pyplot as plt
    import PIL

    # start the plot
    fig, ax = plt.subplots(1)
    ax.set_axis_off()
    img = ax.imshow(image)
    plt.subplots_adjust(wspace=0.05, hspace=0, left=0, bottom=0, right=1, top=1)

    t_default = 500
    default_target = np.array(image)
    default_target[dist < t_default] = (255, 255, 255)

    def update(t=163,y=250):
        # redo threshold
        image_array = np.array(image)
        image_array[dist > t] = (255, 255, 255)
        image_array[y:,:] = (255, 255, 255)
        image_array[y,:] = (255, 0, 0)
        img.set_data(image_array)
        fig.canvas.draw()

    interact(update,
            t=(0, 500, 1),
            y=(200, 300, 1))

# Render Default Configuration

TODO(max) add code to render the un cropped view (closed gripper) with default configuration to see if we are already calibrated.


In [ ]:
render = env.render()
fig, ax = plt.subplots(1,3)
for x in ax :x.set_axis_off()
ax[0].set_title("Threshold Image")
ax[0].imshow(masked_image)
ax[1].imshow(img_mask)
ax[1].set_title("gripper=yellow?")
ax[2].set_title("Rendered Clean")
ax[2].imshow(render)

plt.show()

# Manual Alignment

This code defines functions for parameterized rendering, then opens an interactive plot.
This can be used to find a good range of parameteres to set bound for the optimization later on.


In [ ]:
import types
import numpy as np

#print(image.size)
#image_width, image_height = image.size
image_width, image_height = image.shape[:2]
image_array = image.copy()

def get_image(pos=np.array([0,0,0]),
              orn=np.array([0,0,0]),
              gripper=None,
              fov=40,
              width=image_width,
              height=image_height):
    
    # set the robot calibration
    env.params.sample["calib_pos"] = pos
    env.params.sample["calib_orn"] = orn
    env.params.sample["cam_fov"] = fov
    env.params.sample["gripper_joint_ul"] = gripper
    
    if gripper is not None:
        env.robot.gripper.reset_pose()
    
    env._render_width = width
    env._render_height = height
    env.camera._base_proj_matrix = env.p.computeProjectionMatrixFOV(
        fov=fov, aspect=float(env._render_width)/env._render_height,
        nearVal=0.01, farVal=100.0)
    return env.render()

def get_image_arr(arr,**argv):     
    pos = np.array(arr[0:3])
    orn = np.array(arr[3:6])
    gripper = arr[6]
    fov = arr[7]
    return get_image(pos, orn, gripper, fov, **argv)

# normalized intersection over union function
def niou_func(rnd):
    rnd_mask = np.any(rnd<100, axis=2) 
    intersection = np.sum(rnd_mask * img_mask)
    union = np.sum(rnd_mask + img_mask)
    niou = 1 - intersection / union
    return niou

minscore = 1e30

def testfunc(arr):
    rnd = get_image_arr(arr)
    #norm = np.linalg.norm(rnd-default_target)
    norm = niou_func(rnd)
    #print
    global minscore
    if norm < minscore:
        print(minscore, "[", end='')
        print(*arr, sep=", ", end='')
        print("]")
        minscore = norm
    return(norm)


In [ ]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
from math import pi
import matplotlib.pyplot as plt
import PIL

#df = dict(x=-.004, y=0, z=.007, a=0, b=0, c=0, gripper=0.0455, fov=42)
default_arr = [-8.11760879e-04,  5.91212989e-03,  4.63097354e-04,  2.08804174e-02, 1.21849894e-02,  1.49459515e-02,  4.50389914e-02,  4.40690316e+01]
#default_arr = [-5.93110931e-04,  4.87719789e-03, -1.15584238e-06,  2.63053764e-02, 1.27284190e-02,  1.57081386e-02,  4.46852048e-02,  4.35229641e+01]

df = dict(zip(('x','y','z','a','b','c','gripper','fov'), default_arr))

# start the plot
fig, ax = plt.subplots(1)
ax.set_axis_off()
line = ax.imshow(np.zeros(masked_image.shape))
plt.subplots_adjust(wspace=0.05, hspace=0, left=0, bottom=0, right=1, top=1)

def update(x=df['x'], y=df['y'], z=df['z'],
           a=df['a'], b=df['b'], c=df['c'],
           gripper=df['gripper'], fov=df['fov']):
    
    pos = np.array([x, y, z])
    orn = np.array([a, b, c])
    rnd = get_image(pos, orn, gripper, fov)
    #PIL.Image.fromarray(rnd).save("rendered.png")
    
    # redo threshold
    mix = (masked_image * .5 + rnd * .5).round().astype(np.uint8)
    
    line.set_data(mix)   
    fig.canvas.draw()
    
    niou = niou_func(rnd)
    print("score niou", niou)
    print("score l2", np.linalg.norm(rnd-image_array))

# these are steps
interact(update,
        x=(-.03, .03, .001),
        y=(-.03, .03, .001),
        z=(-.02, .04, .001),
        
        a=(-0.05, 0.05, .001),
        b=(-0.05, 0.05, .001),
        c=(-0.05, 0.05, .001),
        gripper=(.04, .05, .001),
        fov = (41, 45, 0.1));

In [ ]:
print(testfunc(default_arr))

# Optimization

This code runs the differential evolution optimization, usually takes a while ~15min.

In [ ]:
global minscore
minscore = 1e30

from scipy.optimize import differential_evolution
from scipy.optimize import minimize

bounds = [(-.03, .03),  # x
          (-.03, .03),  # y
          (-.02, .04),  # z
          (-0.05, 0.05),  # a
          (-0.05, 0.05),  # b
          (-0.05, 0.05),  # c
          (.04, .05),  # gripper_joint_ul
          (41, 45)]

result = differential_evolution(testfunc, bounds,
                                maxiter=75000, polish=True)

#x0 = [np.mean(x) for x in bounds]
#result = minimize(testfunc, x0=x0, method="Nelder-Mead")

print("\nOptimization done!")
print(result.x, result.fun)

In [ ]:
[-1.34050847e-03, -1.22241527e-03,  1.94246598e-03,  1.83346818e-02,
  1.47678984e-02,  1.13706384e-02,  X,  X]

[-1.58132880e-03 -1.65116112e-03 -2.10224659e-04  3.20026306e-02
  1.15026284e-02  1.16134438e-02  X  X] 0.09725906277630414

[-1.58417016e-03  4.77775224e-03  2.82711839e-04  2.50235106e-02
  9.72836616e-03  1.14202937e-02  X  4.35847151e+01] 0.09756097560975607

XXX Small Res
[-4.04621259e-04  3.56216508e-03  3.36449559e-04  2.58699097e-02
  1.54623827e-02  1.60939934e-02  4.46233639e-02  4.32280177e+01] 0.10060711188204685

XXX High Res run 1 & 2
0.034534877088918 [-7.32462050e-04  5.59790929e-03  4.51445027e-04  2.46986871e-02
  1.14995637e-02  1.53770490e-02  4.49168715e-02  4.39039437e+01]

0.03515413737155215 [-5.93110931e-04,  4.87719789e-03, -1.15584238e-06,  2.63053764e-02,
  1.27284190e-02,  1.57081386e-02,  4.46852048e-02,  4.35229641e+01]

Run ?? 

[-8.11760879e-04,  5.91212989e-03,  4.63097354e-04,  2.08804174e-02,
        1.21849894e-02,  1.49459515e-02,  4.50389914e-02,  4.40690316e+01]

[-8.25197132e-04,  8.80486131e-03, -8.23005900e-04,  2.89505485e-02,
  1.16251534e-02,  1.46831751e-02,  4.48741467e-02,  4.45902255e+01]

In [ ]:
default_arr = [0.011284435936295039, 0.02998802466639384, -0.00833947711031245, -0.04873989515666205, 0.031239671009381145, 0.04826560796124244, 0.040152562068615186, 41.774574723911286]
print(testfunc(default_arr))

default_arr = [-8.11760879e-04,  5.91212989e-03,  4.63097354e-04,  2.08804174e-02, 1.21849894e-02,  1.49459515e-02,  4.50389914e-02,  4.40690316e+01]
print(testfunc(default_arr))

#default_arr = [-5.93110931e-04,  4.87719789e-03, -1.15584238e-06,  2.63053764e-02, 1.27284190e-02,  1.57081386e-02,  4.46852048e-02,  4.35229641e+01]
#print(testfunc(default_arr))

default_arr = [-0.00833239907437136, 0.029619700487285492, -0.019568588856726285, -0.04760457876766309, -0.010309110682057443, -0.03115142369765686, 0.04370905410829578, 41.001146865380086]
print(testfunc(default_arr))

#default_arr = [-8.25197132e-04,  8.80486131e-03, -8.23005900e-04,  2.89505485e-02,  1.16251534e-02,  1.46831751e-02,  4.48741467e-02,  4.45902255e+01]
#default_arr = [-0.0008392997212615084, 0.008794301929382159, -0.0008297287274300752, 0.028990369747127435, 0.011486481116888906, 0.014602577989890647, 0.04488457868564247, 44.59207289541456]
#default_arr = [-0.0010442705210949464, 0.0051371498304054524, -0.0017210247957249432, 0.01877539567147731, 0.01970857299167643, 0.009477141233107732, 0.044506577350280574, 43.59444713296427]
default_arr = [0.00012021346287341083, 0.004711761531908094, -0.0007818706874626134, 0.04217865640212694, 0.02944752036043126, 0.02529005601937725, 0.04434623212006212, 43.406409868283404]
print(testfunc(default_arr))


# Fine-Tune with gradient descent

In [ ]:
result = minimize(testfunc, x0=default_arr, method="Nelder-Mead") 
#result = minimize(testfunc, x0=arr, method="L-BFGS-B") 
print(result)

In [ ]:
import json
import datetime
dt_now = datetime.datetime.now().__str__()

arr = result.x.tolist()
pos = arr[0:3]
orn = arr[3:6]
gripper = arr[6]
fov = arr[7]

rest_pose = [-1.57079633, 0.6981317, 0, -1.3962634, 0, 1.04719755, 0, 0, 0.0400, 0.0400, 0.0000]
calibration_dict = dict(cam_fov=fov, cam_pos=pos, cam_orn=orn, 
                        gripper_joint_ul=gripper,
                        rest_pose=rest_pose, datetime=dt_now)

json_str = json.dumps(calibration_dict, indent=4, separators=(',', ': '))
print(json_str)

In [ ]:
save_calib = True

if save_calib:
    calib_latest_path = "/home/kuka/lang/robot/gym_grasping/gym_grasping/robots/models/kuka_iiwa/calibration_latest.json"

    with open(calib_latest_path, "w") as fo:
        json.dump(calibration_dict, fo, indent=4, separators=(',', ': '))
        fo.write("\n")
    print("saved calibration file.")

# Verify Results

In [ ]:
plt.ioff()
fig, ax = plt.subplots(1)
plt.subplots_adjust(wspace=0.05, hspace=0, left=0, bottom=0, right=1, top=1)
ax.set_axis_off()

render # f
rnd = get_image_arr(arr)
#niou = niou_func(rnd)
#print(niou)
#rnd = render
mix = (masked_image * .5 + rnd * .5).round().astype(np.uint8)

ax.imshow(mix)
ax.text(100, 300, "combined image",  fontsize=14, verticalalignment='top')
plt.show()

In [ ]:
env_calibrated = RobotSimEnv(task='stackVel', renderer='egl', max_steps=200,
                             initial_pose='close', gripper_delay=0,
                             gripper_act_type="continuous",
                             param_randomize=False,
                             img_size=image_size)

for objIds in (env_calibrated._task.objects, env_calibrated._task.surfaces):
    print(objIds)
    for objId in objIds:
        p.removeBody(objId, physicsClientId=env_calibrated.cid)


In [ ]:
print(dt_now)
rnd_env = env_calibrated.render()

plt.ioff()
fig, ax = plt.subplots(1)
plt.subplots_adjust(wspace=0.05, hspace=0, left=0, bottom=0, right=1, top=1)
ax.set_axis_off()

mix = (masked_image * .5 + rnd_env * .5).round().astype(np.uint8)

ax.imshow(mix)
ax.text(100, 300, "combined image",  fontsize=14, verticalalignment='top')
plt.show()

# Resize View

While we want to do the optimization with a full view of the image, we usually want to do RL on small resized images.

In [ ]:
def make_square(image):
    if isinstance(image, np.ndarray):
        image = PIL.Image.fromarray(image)
    width, height = image.size
        
    assert(width > height)
    border = (width - height)//2
    # left, upper, right, lower
    box = (border, 0, width-border, height)  
    image_square = image.crop(box) 
    return image_square

image_square = make_square(image)
rendered = get_image_arr(arr)
render_square = make_square(rendered)
w_sq,h_sq = image_square.size
rendered_small = get_image_arr(arr,width=w_sq,height=h_sq)

# print
print("rendered image size", image.size, "after cropping", image_square.size)
print("rendered small image size", rendered_small.shape)

# plot
fig, ax = plt.subplots(1,3)
for x in ax: x.set_axis_off()
fig.suptitle("Cropped Images (second and third should be same)")
ax[0].imshow(image_square)
ax[0].set_title("cropped camera")
ax[1].imshow(render_square)
ax[1].set_title("cropped render")
ax[2].imshow(rendered_small)
ax[2].set_title("render small")
plt.show()

render_cmb = (np.array(render_square) - np.array(rendered_small))
print("sum of differences", render_cmb.sum())


## Camera Intrinsics

We can compare the factory calibration fov to the value we found. This can be used to validate the quality of the calibration.

TODO(add code for getting camera calibration from API)

In [ ]:
from math import atan, pi
instrinsics = dict(
    width = 1280,
    height = 720,
    ppx = 632.806,
    ppy = 368.559,
    fx = 926.835,
    fy = 926.835)

fov_v = 2*atan(instrinsics["width"]/(2*instrinsics["fx"]))*180/pi 
fov_h = 2*atan(instrinsics["height"]/(2*instrinsics["fy"]))*180/pi

print("ppx_deviation=",instrinsics["width"]/2-instrinsics["ppx"])
print("ppy_deviation=",instrinsics["height"]/2-instrinsics["ppy"])
print()

print("fov v =", fov_v)
print("fov h =", fov_h)

fov_calib = arr[7]
print("fov c =",fov_calib )


factory_deviation = (fov_h - fov_calib)/  fov_h
print("Deviation to factory: {}[%]".format(factory_deviation * 100))